# Machine Learning Homework 2: Naive Bayes

### 1. Load the data into a Python data frame.

In [1]:
# Import module
import numpy as np
import pandas as pd
import re
import time
import functools as ft

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from re import sub

In [8]:
data = pd.read_table("SMSSpamCollection.txt", header=None) 
# make sure to put the '.txt' file in the work directory and name it correctly
data.head()

,0,1
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


###  2. Pre-process the SMS messages: Remove all punctuation and numbers from the SMS messages, and change all messages to lower case.

In [12]:
data.columns = ["type", "text"]
r = '[^A-Za-z\s]+'
text = data.iloc[:,1].apply(lambda x: re.sub(r, "", x).lower().strip())
data.text = text
data.head()

,type,text
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in a wkly comp to win fa cup final ...
3,ham,u dun say so early hor u c already then say
4,ham,nah i don t think he goes to usf he lives arou...


### 3.  Shuffle the messages and split them into a training set (2,500 messages), a validation set (1,000 messages) and a test set (all remaining messages).

In [13]:
# Shuffle dataset
data = shuffle(data, random_state=12345)

In [14]:
# Create train, validation and test dataset
train_set = data.iloc[0:2500,:]
validation_set = data.iloc[2500:3500,:]
test_set = data.iloc[3500:, :]

In [18]:
train_spam = list(train_set.loc[data.type == 'spam', :].text)
train_ham = list(train_set.loc[data.type == 'ham', :].text)
train_message = list(train_set.text)
train_label = list(train_set.type)
validation_message = list(validation_set.text)
validation_label = list(validation_set.type)
test_message = list(test_set.text)
test_label = list(test_set.type)

### 4.  Build Naive Bayes Classfier

In [17]:
class NaiveBayesForSpam:
    def train(self, hamMessages, spamMessages) :
        self.words = set (' '.join(hamMessages + spamMessages).split())
        self.priors = np.zeros (2)
        self.priors [0] = float(len(hamMessages)) / (len(hamMessages)+len(spamMessages))
        self.priors[1] = 1.0 - self.priors[0]
        self.likelihoods = []
        for i , w in enumerate (self.words):
            prob1 = (1.0 + len ([m for m in hamMessages if w in m])) / len(hamMessages)
            prob2 = (1.0 + len([m for m in spamMessages if w in m]) ) / len(spamMessages)
            self.likelihoods.append([min(prob1, 0.95), min(prob2, 0.95)])
        self.likelihoods = np.array(self.likelihoods).T
        
    def train2 (self, hamMessages, spamMessages) :
        self.words = set (" ".join(hamMessages + spamMessages).split())
        self.priors = np. zeros (2)
        self.priors[0] = float (len(hamMessages)) / (len(hamMessages) + len(spamMessages))
        self.priors [1] = 1.0 - self.priors[0]
        self.likelihoods = [ ]
        spamkeywords = [ ]
        for i , w in enumerate(self.words):
            prob1 = (1.0 + len([m for m in hamMessages if w in m])) / len(hamMessages)
            prob2 = (1.0 + len([m for m in spamMessages if w in m])) / len(spamMessages)
            if prob1 * 20 < prob2 :
                self.likelihoods.append([min(prob1, 0.95), min(prob2, 0.95)])
                spamkeywords.append(w)
        self.words = spamkeywords
        self.likelihoods = np.array(self.likelihoods).T
        
    def predict(self, message) :
        posteriors = np.copy (self.priors)
        for i , w in enumerate (self.words) :
            if w in message.lower() : # convert to lower−case, delete it
                posteriors *= self.likelihoods[:, i]
            else:
                posteriors *= np.ones(2) - self.likelihoods[:, i]
            posteriors = posteriors / np. linalg.norm(posteriors, ord = 1) # normalise
        if posteriors[0] > 0.5:
            return ['ham', posteriors[0]]
        return ['spam', posteriors[1]]
    
    def score(self, messages, labels) :
        confusion = np.zeros(4).reshape(2 ,2)
        for m, l in zip (messages, labels) :
            if self.predict(m)[0] == 'ham' and l == 'ham':
                confusion[0 ,0] += 1
            elif self.predict(m)[0] == "ham" and l == "spam":
                confusion[0,1] += 1
            elif self.predict(m)[0] == "spam" and l == "ham":
                confusion[1,0] += 1
            elif self.predict(m)[0] == "spam" and l == "spam":
                confusion[1,1] += 1
        return (confusion[0,0] + confusion[1,1]) / float(confusion.sum()), confusion
    
    # define new function to reduce the false positives
    def predict2(self, message) :
        posteriors = np.copy (self.priors)
        for i , w in enumerate (self.words) :
            if w in message.lower() : # convert to lower−case, delete it
                posteriors *= self.likelihoods[:, i]
            else:
                posteriors *= np.ones(2) - self.likelihoods[:, i]
            posteriors = posteriors / np. linalg.norm(posteriors, ord = 1) # normalise
        if posteriors[0] > 0.2:
            return ['ham', posteriors[0]]
        return ['spam', posteriors[1]]
    
    def score2(self, messages, labels) :
        confusion = np.zeros(4).reshape(2 ,2)
        for m, l in zip (messages, labels) :
            if self.predict2(m)[0] == 'ham' and l == 'ham':
                confusion[0 ,0] += 1
            elif self.predict2(m)[0] == "ham" and l == "spam":
                confusion[0,1] += 1
            elif self.predict2(m)[0] == "spam" and l == "ham":
                confusion[1,0] += 1
            elif self.predict2(m)[0] == "spam" and l == "spam":
                confusion[1,1] += 1
        return (confusion[0,0] + confusion[1,1]) / float(confusion.sum()), confusion

### 5. What is the purpose of each function? What do ’train’ and ‘train2’ do, and what is the difference between them? Where in the code is Bayes’ Theorem being applied?

 - Both functions 'train' and 'train2' functions calculate the prior probability of $P(HAM)$ and $P(SPAM)$ and the likelihood of $P(word \mid HAM)$ and $P(word \mid SPAM)$ based on a lists of words in the 'hamMessages' and the 'spamMessages'. Moreover, both functions use the Laplace Estimator in the loop, changing all probabilities of zero to 0.05. By doing so, we could sort out HAM and SPAM by comparing the $P(HAM \mid new word)$ = priors[0]$\times$prob_1 to $P(SPAM \mid new word)$ = priors[1]$\times$prob_2 in the 'predict' function. The 'score' function calculates the confusion matrix and accuracy by comparing the classifications of each new message with the original labels.

- The difference between train and train2 is mainly in how it treats words classified as spam words. The line of code ```if prob1$\times$20 < prob2``` assigns a higher weight to the likelihood of a message being 'HAM' by multiplying it by 20. As a result, the spam keywords in 'train2' have a much higher likelihood of appearing in a spam message. 


- Bayes' Theorem is applied in the predict and predict2 functions. The `posteriors` variable stores the proportional probabilities that the new message is ham and spam respectively. This is calculated by multiplying the prior of ham and spam (self.priors) by the likelihood of the message given that it is ham and spam. This is the application of Bayes' Theorem. Take for example the probability that a new message is ham (where spamword1, spamword3...spamwordn is in new message, but spamword2 is not): $P(HAM \mid word_1 \cap not word_2 \cap word_3..... \cap word_n)$. By Bayes' Theorem, the code breaks up this probability into likelihood*prior = $P(word_1 \mid HAM)$$\times$$P(not word_2 \mid HAM)$$\times$$P(word_3 \mid HAM)$$\times$...$\times$$P(word_n \mid HAM)$$\times$$P(HAM)$. Note that above, we have broken the likelihood up further by making the Naive Bayes class conditional independence assumption. In the code this product of likelihoods and priors is computed with a simple for-loop that iterates through all spam key words and updates `posteriors` at each spam key word. 

- These posterior probabilites are only proportional to the true probabilities as we have ignored the marginal probability of the new message. Therefore we must scale these probabilities by normalization as though we would still get the same classification, the probabilites would become very small which would cause programming issues. Now with our threshold for classification, we can predict whether the new sentence is spam or ham. 


### 6. Use your training set to train the classifiers ‘train’ and ‘train2’.

In [19]:
# Model 'NB_classifer' use train
NB_classifer = NaiveBayesForSpam()
NB_classifer.train(hamMessages=train_ham, spamMessages=train_spam)

# Model 'NB_classifer2' use train2
NB_classifer2 = NaiveBayesForSpam()
NB_classifer2.train2(hamMessages=train_ham, spamMessages=train_spam)

In [ ]:
start_time = time.time()
train_score_train = NB_classifer.score(messages = train_message, labels = train_label)
print("Running time for 'train'  function through train set     : %.4s seconds" % (time.time() - start_time))

start_time = time.time()
train_score_train2 = NB_classifer2.score(messages = train_message, labels = train_label)
print("Running time for 'train2' function through train set     : %.4s seconds\n" % (time.time() - start_time))

start_time = time.time()
val_score_train = NB_classifer.score(messages = validation_message, labels = validation_label)
print("Running time for 'train'  function through validation set: %.4s seconds" % (time.time() - start_time))

start_time = time.time()
val_score_train2 = NB_classifer2.score(messages = validation_message, labels = validation_label)
print("Running time for 'train2' function through validation set: %.4s seconds" % (time.time() - start_time))

In [25]:
result_train = train_score_train
accuracy_train, confusion_mat_train = result_train
confusion_mat_train = pd.DataFrame(confusion_mat_train)
confusion_mat_train.columns = ['Actual_HAM', 'Actual_SPAM']
confusion_mat_train.index = ['Predicted_HAM', 'Predicted_SPAM']
print("Result for 'train' function through the training set\n")
print("Accuracy: %.4f\n" %(accuracy_train))
print("************Confusion_Matrix************\n", confusion_mat_train)

Result for 'train' function through the training set

Accuracy: 0.9656

************Confusion_Matrix************
                 Actual_HAM  Actual_SPAM
Predicted_HAM       2098.0         31.0
Predicted_SPAM        55.0        316.0


In [84]:
result_train2 = train_score_train2
accuracy_train2, confusion_mat_train2 = result_train2
confusion_mat_train2 = pd.DataFrame(confusion_mat_train2)
confusion_mat_train2.columns = ['Actual_HAM', 'Actual_SPAM']
confusion_mat_train2.index = ['Predicted_HAM', 'Predicted_SPAM']
print("Result for 'train2' function through the training set\n")
print("Accuracy: %.4f\n" %(accuracy_train2))
print("************Confusion_Matrix************\n", confusion_mat_train2)

Result for 'train2' function through the training set

Accuracy: 0.9752

************Confusion_Matrix************
                 Actual_HAM  Actual_SPAM
Predicted_HAM       2145.0         54.0
Predicted_SPAM         8.0        293.0


In [85]:
result_val= val_score_train
accuracy_val, confusion_mat_val = result_val
confusion_mat_val = pd.DataFrame(confusion_mat_val)
confusion_mat_val.columns = ['Actual_HAM', 'Actual_SPAM']
confusion_mat_val.index = ['Predicted_HAM', 'Predicted_SPAM']
print("Result for 'train' function through the validation set\n")
print("Accuracy: %.4f\n" %(accuracy_val))
print("************Confusion_Matrix************\n", confusion_mat_val)

Result for 'train' function through the validation set

Accuracy: 0.9680

************Confusion_Matrix************
                 Actual_HAM  Actual_SPAM
Predicted_HAM        858.0         13.0
Predicted_SPAM        19.0        110.0


In [86]:
result_val2 = val_score_train2
accuracy_val2, confusion_mat_val2 = result_val2
confusion_mat_val2 = pd.DataFrame(confusion_mat_val2)
confusion_mat_val2.columns = ['Actual_HAM', 'Actual_SPAM']
confusion_mat_val2.index = ['Predicted_HAM', 'Predicted_SPAM']
print("Result for 'train2' function through the validation set\n")
print("Accuracy: %.4f\n" %(accuracy_val2))
print("************Confusion_Matrix************\n", confusion_mat_val2)

Result for 'train2' function through the validation set

Accuracy: 0.9740

************Confusion_Matrix************
                 Actual_HAM  Actual_SPAM
Predicted_HAM        874.0         23.0
Predicted_SPAM         3.0        100.0


### 7.  Using the validation set, explore how each of the two classifiers performs out of sample.

- The accuracy of classifications from the 'train2' function is 97.4%, which is 0.6% higher than classifications using the 'train' function on the same validation set. In the confusion matrix, we can see that there are more true positives (i.e. actual ham messages predicted as ham messages) and less false positives (i.e. actual ham messages predicted as spam messages). However, the 'train2' function has worse performance when it comes to predicting spam messages (classifying the actual spam messages as predicted spam messages), which is the trade-off between 'train' and 'train2'.

### 8. Why is the ‘train2’ classifier faster? Why does it yield a better accuracy both on the training and the validation set?

In [100]:
print("The length of words through NB_classifier:", len(NB_classifer.words))
print("The length of words through NB_classifier2:", len(NB_classifer2.words))

The length of words through NB_classifier: 5157
The length of words through NB_classifier2: 198


- Running time for 'train'  function through train set     : 309. seconds
- Running time for 'train2' function through train set     : 11.4 seconds
- Running time for 'train'  function through validation set: 113. seconds
- Running time for 'train2' function through validation set: 4.02 seconds


- The 'train2' classifier is approximately 30 times faster than the 'train1 because the 'predict' function makes use of the list of "spam key words" from the 'train2' function, which satisfies the condition on $P(word \mid HAM)\times20   < P(word \mid SPAM)$, which makes a lot smaller a set of the words (198) for filtering SPAM. On the other hand, the classifier based on the 'train' function has to use the whole list of SPAM and HAM words (5157), which takes  30 times longer to train.     

### 9.  How many false positives did you get in your validation set? How would you change the code to reduce false positives at the expense of possibly having more false negatives?

In [87]:
# False positves for train2 at threshold = 0.5
fp_train2 = val_score_train2[1][1,0]
print(int(fp_train2))

3


In [88]:
val_score_train2_reduced = NB_classifer2.score2(messages = validation_message, labels = validation_label)

In [89]:
# False positives after decreasing threshold to 0.3
fp_train2_reduced = val_score_train2_reduced[1][1,0]
print(int(fp_train2_reduced))

1


The accuracy of the 'train2' classifier on the test set is 96.96%, which is 0.44% lower than the accuracy of the 'train2' classifier on the validation set. The percentage of true positives is a bit higher in the validation set (87.4% compared to 86.29% in the test set). However, true negatives are better predicted on the test set (0.67% higher). All in all, the outcome from the test set is similar to the outcome from the validation set.

### 10. Run the ‘train2’ classifier on the test set and report its performance using a confusion matrix.

In [90]:
result_test = NB_classifer2.score(messages = test_message, labels = test_label)

In [91]:
accuracy_test, confusion_mat_test = result_test
confusion_mat_test = pd.DataFrame(confusion_mat_test)
confusion_mat_test.columns = ['Actual_HAM', 'Actual_SPAM']
confusion_mat_test.index = ['Predicted_HAM', 'Predicted_SPAM']
print("Result for 'train2' function through the test set\n")
print("Accuracy: %.4f\n" %(accuracy_test))
print("************Confusion_Matrix************\n", confusion_mat_test)

Result for 'train2' function through the test set

Accuracy: 0.9696

************Confusion_Matrix************
                 Actual_HAM  Actual_SPAM
Predicted_HAM       1788.0         56.0
Predicted_SPAM         7.0        221.0


- The accuracy of classifications assigned using the 'train2' classfier on the test set is 96.96%, which is 0.44% lower than the 'train2' classifier on the validation set. Furthermore, there are only 7 messages classified as false positives in the confusion matrix, which is a similiar outcome from the validation set. 